In [1]:
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
import cufflinks as cf
import colorlover as cl
import bokeh
import numpy as np
from bokeh.io import output_file, show
from bokeh.models import HoverTool
from bokeh.plotting import figure

In [2]:
plotly.tools.set_credentials_file(username='ryanzhao9459', api_key='0rWuTS32zBMaefc0mWUj')

In [3]:
file = pd.read_csv('match.csv')

In [4]:
file_1 = pd.read_csv('team_performance.csv')

In [5]:
file_2 = pd.read_csv('players_age_wiki.csv')

In [6]:
file_3 = pd.read_csv('2018_ages.csv')

In [7]:
file_4 = pd.read_csv('2019_ages.csv')

## Data processing

heatmap

In [8]:
team_name_rank = ['Royal Challengers Bangalore','Kings XI Punjab','Mumbai Indians','Kolkata Knight Riders',\
                 'Chennai Super Kings','Delhi Capitals','Rajasthan Royals','Sunrisers Hyderabad']
table = file.loc[:,['team1','team2','winner']]
grp = table.groupby('team1')

In [9]:
def sort_name(team_name):
    team_name_sort = []
    for i in range(len(team_name)):
        l1 = team_name[i].split()
        lala = l1[0] + ' '
        for j in range(1,len(l1)):
            lulu = l1[j][0]
            lala = lala + lulu+'.'
        a = ''.join(lala)
        team_name_sort.append(a)
    return team_name_sort

In [10]:
team_name_sort =sort_name(team_name_rank)

In [11]:
def win_heatmap(grp,team_name):
    
    result = [[0]*8 for x in range(8)]
    total_win = []

    for (i,m) in enumerate(team_name):
        grp1 = grp.get_group(m)
        grp2 = grp1.groupby('team2')
        p = 0
        q = 0
        for (j,n) in enumerate(team_name):
            grp4 = grp.get_group(n)
            grp5 = grp4.groupby('team2')
            num_1 = list(grp1['team2']).count(n)
            num_2 = list(grp4['team2']).count(m)
            if m == n or num_1 == 0 or num_2 == 0:
                result[i][j] = '-'
            else:
                grp3 = grp2.get_group(n)
                grp6 = grp5.get_group(m)
                result[i][j] = (list(grp3['winner']).count(m)+list(grp6['winner']).count(m))/ \
                                        (len(list(grp3['winner']))+len(list(grp6['winner'])))
                p = p + list(grp3['winner']).count(m)+list(grp6['winner']).count(m)
                q = q + len(list(grp3['winner']))+len(list(grp6['winner']))
        total_win.append(p/q)
    return result,total_win

In [12]:
result ,total_win = win_heatmap(grp,team_name_rank)

bar(toss)

In [13]:
toss_decisions = file[['toss_winner', 'toss_decision', 'id']].\
                        groupby(['toss_winner', 'toss_decision']).agg({'id': 'count'})

toss_decisions.head(3)
toss_decisions = toss_decisions.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))

In [14]:
sb3 = pd.DataFrame(data = total_win,columns=['Win rate total'],index = team_name_rank)
sb3.sort_index(inplace=True)
sb3 = sb3.apply(lambda x: 100 * x)

line(rank)

In [15]:
def rank(file):
    team_name = ['Royal Challengers Bangalore','Kings XI Punjab','Mumbai Indians','Kolkata Knight Riders',\
                 'Chennai Super Kings','Delhi Capitals','Rajasthan Royals','Sunrisers Hyderabad']
    year = sorted(list(set(file['year'])))
    grp = file.loc[:,['year','Team']]
    grp1 = grp.groupby('year')
    result = [[0]*len(year) for x in range(len(team_name))]

    for i in range(len(year)):
        grp2 = grp1.get_group(year[i])
        for j in range(len(team_name)):
            if team_name[j] in list(grp2['Team']):
                result[j][i] = list(grp2['Team']).index(team_name[j]) + 1
            else:
                result[j][i] = '-'
    return result

In [16]:
result_rank = rank(file_1)

age

In [17]:
short_names = {'Chennai Super Kings':'CSK', 'Delhi Capitals': 'DD', 'Kings XI Punjab' :'KXIP',
              'Mumbai Indians':'MI', 'Rajasthan Royals':'RR', 'Royal Challengers Bangalore':'RCB',
              'Sunrisers Hyderabad': 'SRH', 'Kolkata Knight Riders': 'KKR'}
new_dict = {v:k for k,v in short_names.items()}

In [18]:
table_age = file_2.loc[:,['team_name','2018']]
table_age = table_age.drop([0,4,6,10,12,13,14])
team_name_age_1 = list(table_age['team_name'])
grp1 = file_3.groupby('2018')

In [19]:
def age(grp,team):
    result = [[],[]]
    for i in team:
        grp2 = grp.get_group(i)
        result[0].append(min(list(grp2['age_2018'])))
        result[1].append(max(list(grp2['age_2018'])))
    return result

In [20]:
result_age = age(grp1,team_name_age_1)

In [21]:
team_name_age = [new_dict[i] for i in team_name_age_1]

### plot

heatmap

In [22]:
data = [go.Heatmap(z = result,x=team_name_sort,y=team_name_sort,colorscale='Reds')]
layout = go.Layout(
    title = dict(text = 'Head to Head',
                 font = dict(family='Times New Roman, monospace', size=20, color='#000000'),
                 x = 0,
                 xref = 'paper'),
    width = 900, height = 700,
    xaxis = dict(tickfont = dict(family = 'Old Standard TT, serif',
                 size = 12,
                 color = 'black'),
                 tickangle = 30),
    yaxis = dict(tickfont = dict(family = 'Old Standard TT, serif',
                 size = 12,
                 color = 'black'))
)
fig = go.Figure(data = data, layout = layout)
py.iplot(fig)
# url_heat = py.iplot(fig)
# url_heat.resource

C:\Users\ryanz\Anaconda3\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



bar

In [26]:
index = [0,4,7,3,5,2,6,1]
a = list(sb3.index)
a = [a[i] for i in index]
a = sort_name(a)
b = list(sb3.values)
b = [b[i] for i in index]

trace1 = go.Scatter(
    x = a,
    y = b,
    name = 'Winning rate',
    marker = dict(color= 'rgba(219, 64, 82, 1.0)')
)
index = [0,7,-1,6,9,4,-2,2]
b = list(toss_decisions.id)[0::2]
b = [b[i] for i in index]
trace2 = go.Bar(
    x = a,
    y = b,
    name = 'Bat',
    width = [0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3],
    marker = dict(
        color = 'rgba(219, 64, 82, 0.7)',
        line=dict(color='rgba(219, 64, 82, 1.0)',width=2)
    )
)

b = list(toss_decisions.id)[1::2]
b = [b[i] for i in index]

trace3 = go.Bar(
    x = a,
    y = b,
    name = 'Field',
    width = [0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3],
    marker = dict(
        color = 'rgba(55, 128, 191, 0.7)',
        line = dict(color='rgba(55, 128, 191, 1.0)',width=2),
    )
)
#292942
data = [trace1, trace2, trace3]
layout = go.Layout(
    title=dict(text = 'Toss result',
               font = dict(family='Times New Roman, monospace', size=20, color='#000000'),
               x = 0,
               xref = 'paper'
               ),
    xaxis = dict(tickangle = 30,
                 tickfont = dict(family='Times New Roman, monospace', size=12, color='#000000')
                 ),
    yaxis = dict(title = 'Percent',range = [0,100],
                 titlefont = dict(family='Times New Roman, monospace', size=16, color='#000000'),
                 tickfont = dict(family='Times New Roman, monospace', size=12, color='#000000')),
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='bar-line')
# url_bar = py.iplot(fig,filename='bar-line')
# url_bar.resource

rank

In [24]:
data = []
for i in range(len(result_rank)):
    data.append(go.Scatter(
        x = list(range(2008,2020)),
        y = list(result_rank[i]),
        name = team_name_rank[i],
    ))


layout = go.Layout(
  annotations = [
    dict(
      x = 2007.5,
      y = 2,
      showarrow = False,
      text = "Top Four",
      xref = "paper",
      yref = "paper"
    ),
    dict(
      x = 2007.5,
      y = 6,
      showarrow = False,
      text = "trash",
      xref = "paper",
      yref = "paper"
    )
  ],
  showlegend = True,
  title = dict(text = "Ranking",x=0,xref = 'paper',
              font = dict(family='Times New Roman, monospace', size=20, color='#000000')),
  yaxis = dict(range=[10.5, 0]),
  xaxis = dict(showgrid=False)
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='rank')
# url_rank = py.iplot(fig,filename='rank')
# url_rank.resource

age

In [25]:
trace1 = go.Scatter(
                x = team_name_age,
                y = list(table_age['2018']),
                fill = 'tonexty',
                line = dict(width=0.5,color='rgb(131, 90, 241)'),
                mode = 'lines')
trace0 = go.Scatter(
                x = team_name_age,
                y = result_age[0],
                fill = None,
                line = dict(width=0.5,color='rgb(111, 231, 219)'),
                mode = 'lines')
trace2 = go.Scatter(
                x = team_name_age,
                y = result_age[1],
                fill = 'tonexty',
                line = dict(width=0.5,color='rgb(111, 231, 219)'),
                mode = 'lines')
layout = go.Layout(
                title = dict(text = 'Age',
                             font = dict(family='Times New Roman, monospace', size=20, color='#000000'),
                             x = 0,
                             xref = 'paper'),
                yaxis = dict(range=[15,40]))
data = [trace0,trace1,trace2]
fig = go.Figure(data = data,layout = layout)
py.iplot(fig)